In [167]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import nltk, re, spacy, scipy
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
import numpy as np
! pip install -q nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.internals import find_jars_within_path
from nltk.tag import StanfordPOSTagger
from nltk.tag import StanfordNERTagger
from sklearn import linear_model
from sklearn.metrics import fbeta_score, accuracy_score

import os
from lxml import objectify
import pandas as pd
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pprint import pprint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [312]:
dataset = pd.read_csv("/content/drive/MyDrive/All-2479-Answers-retrieved-from-MedQuAD.csv")
df = pd.read_csv("/content/drive/MyDrive/All-qrels_LiveQAMed2017-TestQuestions_2479_Judged-Answers.txt", sep=" ", header=None, 
                 names=["no.","judgement score ", "AnswerID"])
dataset, df

(                         AnswerID  \
 0           ADAM_0003147_Sec1.txt   
 1           ADAM_0003147_Sec2.txt   
 2           ADAM_0002818_Sec2.txt   
 3           ADAM_0002818_Sec7.txt   
 4           ADAM_0002818_Sec9.txt   
 ...                           ...   
 2474  MPlusDrugs_0000553_Sec5.txt   
 2475  MPlusDrugs_0000553_Sec6.txt   
 2476  MPlusDrugs_0000553_Sec8.txt   
 2477  MPlusDrugs_0000553_Sec9.txt   
 2478  MPlusDrugs_0000553_Sec7.txt   
 
                                                  Answer  
 0     Question: What is (are) Polycystic ovary syndr...  
 1     Question: What causes Polycystic ovary syndrom...  
 2     Question: What causes Noonan syndrome ?\nURL: ...  
 3     Question: What are the complications of Noonan...  
 4     Question: How to prevent Noonan syndrome ?\nUR...  
 ...                                                 ...  
 2474  Question: What should I do if I forget a dose ...  
 2475  Question: What are the side effects or risks o...  
 2476  Ques

In [313]:
# Read Text Files with Pandas using read_table()
  
# importing pandas
import pandas as pd
  
# read text file into pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/All-qrels_LiveQAMed2017-TestQuestions_2479_Judged-Answers.txt", sep=" ", header=None, 
                 names=["no.","judgement score ", "answer"])

In [314]:
df

,no.,judgement score,answer
0,1,1-Incorrect,ADAM_0003147_Sec1.txt
1,1,1-Incorrect,ADAM_0003147_Sec2.txt
2,1,2-Related,ADAM_0002818_Sec2.txt
3,1,2-Related,ADAM_0002818_Sec7.txt
4,1,2-Related,ADAM_0002818_Sec9.txt
...,...,...,...
2474,104,2-Related,MPlusDrugs_0000553_Sec5.txt
2475,104,2-Related,MPlusDrugs_0000553_Sec6.txt
2476,104,2-Related,MPlusDrugs_0000553_Sec8.txt
2477,104,2-Related,MPlusDrugs_0000553_Sec9.txt


In [315]:
dataset['Question'] = dataset['Answer'].astype(str)
dataset=dataset.drop(columns=['Answer','AnswerID'])
_,dataset['Question'] = dataset['Question'].str.split(':',1).str
dataset['Question'],_ = dataset['Question'].str.split('\n',1).str
# _,dataset['Answer'] = dataset['Answer'].str.split('Answer:',1).str
dataset['Judge_score']=df['judgement score ']
dataset['ID']=df['no.']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """


In [316]:
dataset

,Question,Answer,Judge_score
0,What is (are) Polycystic ovary syndrome ? (Al...,Polycystic ovary syndrome is a condition in w...,1-Incorrect
1,What causes Polycystic ovary syndrome ? (Also...,PCOS is linked to changes in hormone levels t...,1-Incorrect
2,What causes Noonan syndrome ?,Noonan syndrome is linked to defects in sever...,2-Related
3,What are the complications of Noonan syndrome ?,- Buildup of fluid in tissues of body (lymphe...,2-Related
4,How to prevent Noonan syndrome ?,Couples with a family history of Noonan syndr...,2-Related
...,...,...,...
2474,What should I do if I forget a dose of Glimep...,"Before you start to take glimepiride, ask you...",2-Related
2475,What are the side effects or risks of Glimepi...,This medication may cause changes in your blo...,2-Related
2476,What to do in case of emergency or overdose o...,"In case of overdose, call your local poison c...",2-Related
2477,What other information should I know about Gl...,Keep all appointments with your doctor and th...,2-Related


In [7]:
xmlFile = '/content/drive/MyDrive/TREC-2017-LiveQA-Medical-Test.xml'
from lxml import objectify   
import pandas as pd 
with open(xmlFile, "rb") as fobj:
      xml = fobj.read()
root = objectify.fromstring(xml)

In [8]:
subject = []
message = []
nist_paraphrase = []
focus = []
qtype = []
answer = []

In [9]:
for q in root.getchildren(): 
  for qt in q.getchildren():
    if qt.tag == 'Original-Question':
      for qtt in qt.getchildren():
        if qtt.tag == 'SUBJECT':
          subject.append(qtt.text)
        elif qtt.tag == 'MESSAGE':
          message.append(qtt.text)
    if qt.tag == 'NIST-PARAPHRASE':
      nist_paraphrase.append(qt.text)
    if qt.tag == 'ANNOTATIONS':
      temp_focus = []
      temp_type = []
      for qtt in qt.getchildren():
        if qtt.tag == 'FOCUS':
          temp_focus.append(qtt.text)
        elif qtt.tag == 'TYPE':
          temp_type.append(qtt.text)
      focus.append(temp_focus)
      qtype.append(temp_type)
    if qt.tag == 'ReferenceAnswers':
      temp_answer = []
      for qtt in qt.getiterator('ANSWER'):
        temp_answer.append(qtt.text)
      answer.append(temp_answer)


In [10]:
test_df = pd.DataFrame({'subject':subject,
                        'message':message,
                        'nist_paraphrase':nist_paraphrase,
                        'focus':focus,
                        'qtype':qtype,
                        'answer':answer
                        })


In [13]:
test_df['message']

0      What are the references with noonan syndrome a...
1      Re:NDC# 0115-0672-50 Zolmitriptan tabkets 5mg....
2                                 are they gluten free\t
3       vdrl positive patients please tell me what ar...
4       How much glucose is in my GlucaGen HypoKit ? ...
                             ...                        
99      how does effextor cause ED and what is the mi...
100    How long has this non prescription drug been i...
101           i want to know more about aeortic stenosis
102                 What can cause white cells ti uprate
103    Can you please provide Glimepiride storage & a...
Name: message, Length: 104, dtype: object

In [ ]:
def divide_data(ques_1,i):
    ques_1=dataset[dataset['ID'] == i]
    return ques_1

In [ ]:
for i in range(104):
  divide_data()


In [12]:
dataset.isna().sum()

NameError: ignored

In [319]:
dataset['Answer'][25]

" Summary : Clinitest tablets are used to test how much sugar (glucose) there is in a person's urine. Poisoning occurs from swallowing these tablets.   Clinitest tablets used to be used to check how well a person's diabetes was being controlled. These tablets are rarely used today. They are not meant to be swallowed, but could be taken by accident, since they look like pills.  This article is for information only. Do NOT use it to treat or manage an actual poison exposure. If you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States.\n\nPoisonous Ingredient : The poisonous ingredients in Clinitest tablets are:  - Copper sulfate  - Citric acid  - Sodium hydroxide  - Sodium carbonate\n\nWhere Found : The poisonous ingredients are found in Clinitest tablets.  Other products may also contain these in

In [321]:
dataset[dataset.isna()['Answer']]

,Question,Answer,Judge_score


In [322]:
print(dataset.duplicated().sum())

334


In [323]:
dataset=dataset.drop_duplicates()
dataset.shape

(2145, 3)

In [324]:
dataset['Question'].value_counts()

 What is (are) High Blood Pressure ?                                                                                                                                                      15
 What is (are) Shingles ?                                                                                                                                                                  8
 What is (are) Diabetes ?                                                                                                                                                                  7
 What are the treatments for Prescription and Illicit Drug Abuse ?                                                                                                                         7
 What to do for Foot, leg, and ankle swelling ? (Also called: Swelling of the ankles - feet - legs; Ankle swelling; Foot swelling; Leg swelling; Edema - peripheral; Peripheral edema)     6
                                                       

In [325]:
dataset['Answer'].value_counts()

 Take the missed dose as soon as you remember it. However, if it is almost time for the next dose, skip the missed dose and continue your regular dosing schedule. Do not take a double dose to make up for a missed one.) \n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

We have 1709 different question but we have 1644 answer, it means there is a diffrent questions has the same answer, and there is question has multiple answers.

In [326]:
data=pd.DataFrame()
data['ques']=dataset['Question']
data['ans']=dataset['Answer']
print(data.duplicated().sum())

216


In [293]:
data

,ques,ans
0,What is (are) Polycystic ovary syndrome,Polycystic ovary syndrome is a condition in w...
1,What causes Polycystic ovary syndrome,PCOS is linked to changes in hormone levels t...
2,What causes Noonan syndrome,Noonan syndrome is linked to defects in sever...
3,What are the complications of Noonan syndrome,- Buildup of fluid in tissues of body (lymphe...
4,How to prevent Noonan syndrome,Couples with a family history of Noonan syndr...
...,...,...
2474,What should I do if I forget a dose of Glimep...,"Before you start to take glimepiride, ask you..."
2475,What are the side effects or risks of Glimepi...,This medication may cause changes in your blo...
2476,What to do in case of emergency or overdose o...,"In case of overdose, call your local poison c..."
2477,What other information should I know about Gl...,Keep all appointments with your doctor and th...


In [327]:
data[data.duplicated()]

,ques,ans
11,What causes Noonan syndrome ?,Noonan syndrome is linked to defects in sever...
14,Is Noonan syndrome inherited ? (Also called: ...,How is Noonan syndrome inherited? Noonan synd...
47,Do you have information about Learn about glu...,"Summary : On a gluten-free diet, you do not e..."
55,What is (are) Gluten Sensitivity ? (Also call...,"Gluten is a protein found in wheat, rye, and ..."
131,What causes Laryngeal nerve damage ? (Also ca...,Injury to the laryngeal nerves is uncommon. ...
...,...,...
2427,Is supravalvular aortic stenosis inherited ? ...,This condition is inherited in an autosomal d...
2444,Do you have information about RBC count (Also...,Summary : An RBC count is a blood test that m...
2447,Do you have information about CSF cell count,Summary : A CSF cell count is a test to measu...
2448,Do you have information about T-cell count (A...,Summary : A T-cell count measures the number ...


In [178]:
dataset['Judge_score'].unique()

array(['1-Incorrect', '2-Related', '3-Incomplete', '4-Excellent'],
      dtype=object)

In [179]:
dataset['Judge_score'].value_counts()

1-Incorrect     1132
2-Related        653
3-Incomplete     219
4-Excellent      141
Name: Judge_score, dtype: int64

In [180]:
dataset[dataset['Judge_score'] == '4-Excellent'].shape

(141, 3)

We have 217 duplicate question answer but they have different ranks.

In [181]:
def preprocess(corpus, ques_words, cleaning = True, lemmatization = True, remove = True):

    cleaned_corpus = pd.DataFrame()
    for row in corpus:
        dataa = []
        for word in row.split():
            x1 = re.sub(pattern='[^a-zA-Z]',repl=' ',string=word)
            x2 = x1.lower()
            dataa.append(x2)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(dataa)), ignore_index=True)
    corpus=cleaned_corpus
    corpus.columns = ["entail"]
    
    if remove == True:
        stop = set(stopwords.words('english'))
        for word in ques_words:
            stop.remove(word)
        corpus = [[x for x in x.split() if x not in stop] for x in corpus["entail"]]
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        lem = WordNetLemmatizer()
        corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    corpus = [' '.join(x) for x in corpus]

    return corpus

In [182]:
dataset = dataset.convert_dtypes()
ques = pd.Series(dataset['Question'].tolist()).astype(str)
ans = pd.Series(dataset['Answer'].tolist()).astype(str)

In [183]:
ques_wor = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
dataset['Question'] = preprocess(ques, ques_wor)

In [184]:
ques_wor=[]
dataset['Answer']=preprocess(ans, ques_wor)

In [185]:
dataset

,Question,Answer,Judge_score
0,what polycystic ovary syndrome,polycystic ovary syndrome condition woman imba...,1-Incorrect
1,what cause polycystic ovary syndrome,pcos link change hormone level make harder ova...,1-Incorrect
2,what cause noonan syndrome,noonan syndrome link defect several genes cert...,2-Related
3,what complications noonan syndrome,buildup fluid tissue body lymphedema cystic hy...,2-Related
4,how prevent noonan syndrome,couple family history noonan syndrome may want...,2-Related
...,...,...,...
2474,what forget dose glimepiride,start take glimepiride ask doctor forget take ...,2-Related
2475,what side effect risk glimepiride,medication may cause change blood sugar know s...,2-Related
2476,what case emergency overdose glimepiride,case overdose call local poison control center...,2-Related
2477,what information know glimepiride,keep appointments doctor laboratory fast blood...,2-Related


In [186]:
data_train,data_test = train_test_split(dataset, train_size = 0.5, random_state = 2022)

In [187]:
data_train.shape,data_test.shape

((1072, 3), (1073, 3))

In [188]:
#applying data augmentation
label_1=data_train[data_train['Judge_score'] == '1-Incorrect']
label_2=data_train[data_train['Judge_score'] == '2-Related']
label_3=data_train[data_train['Judge_score'] == '3-Incomplete']
label_4=data_train[data_train['Judge_score'] == '4-Excellent']
label_1.shape,label_2.shape,label_3.shape,label_4.shape

((552, 3), (336, 3), (111, 3), (73, 3))

In [189]:
from sklearn.utils import resample
def resample_fun(label_b,Label_a):
    upsample = resample(label_b,
               replace=True,
               n_samples=len(Label_a),
               random_state=42)
    np.unique(np.array(upsample)[:,2],return_counts=True)
    #add sampled ones part to zeros part again
    return upsample

In [190]:
samp_1=resample_fun(label_2,label_1)
training_1 = pd.concat([label_1,samp_1])
training_1.shape

(1104, 3)

In [191]:
samp_1=resample_fun(label_3,label_1)
training_2 = pd.concat([training_1,samp_1])
training_2.shape

(1656, 3)

In [192]:
samp_1=resample_fun(label_4,label_1)
training_3 = pd.concat([training_2,samp_1])
training_3.shape

(2208, 3)

In [193]:
training_3

,Question,Answer,Judge_score
832,information saline nasal wash also call salt w...,na,1-Incorrect
2152,what cataract,cataract cloud lens eye affect vision cataract...,1-Incorrect
927,what research clinical trials do ohtahara synd...,ninds conduct support extensive research progr...,1-Incorrect
2099,what cause miscarriage threaten,pregnant women vaginal bleed without abdominal...,1-Incorrect
594,what symptoms drug allergies,common symptoms drug allergy include hive itch...,1-Incorrect
...,...,...,...
2276,how diagnose liver cancer hepatocellular carci...,thedoctor perform physical exam ask symptoms p...,4-Excellent
500,what complications beckwith wiedemann syndrome,complications occur development tumors feed pr...,4-Excellent
131,what cause laryngeal nerve damage,injury laryngeal nerve uncommon occur complica...,4-Excellent
2201,what foot leg ankle swell,painless swell may affect legs may include cal...,4-Excellent


In [271]:
data_set=pd.DataFrame()
data_set=pd.concat([training_3,data_test]) #after vectorizing and toknize we will split them agin into 2208 train and 1073 test

In [272]:
data_set

,Question,Answer,Judge_score
832,information saline nasal wash also call salt w...,na,1-Incorrect
2152,what cataract,cataract cloud lens eye affect vision cataract...,1-Incorrect
927,what research clinical trials do ohtahara synd...,ninds conduct support extensive research progr...,1-Incorrect
2099,what cause miscarriage threaten,pregnant women vaginal bleed without abdominal...,1-Incorrect
594,what symptoms drug allergies,common symptoms drug allergy include hive itch...,1-Incorrect
...,...,...,...
1956,how prevent thyroid cancer,know prevention awareness risk previous radiat...,2-Related
2261,what cause gastrointestinal bleed,gi bleed may due condition serious include ana...,1-Incorrect
212,need see doctor perianal streptococcal cellulitis,call health care provider child complain pain ...,2-Related
642,how phentermine topiramate use what dosage,phentermine topiramate come extend release cap...,3-Incomplete


In [ ]:
# data_set['Answer'] = data_set['Answer'].apply(lambda x:  nltk.word_tokenize(x))
# data_set['Question'] = data_set['Question'].apply(lambda x:  nltk.word_tokenize(x))
# data_set['Answer'],data_set['Question']

In [273]:
def convert(list):
    return tuple(list)
ans_vec=convert(data_set['Answer'])
ques_vec=convert(data_set['Question'])

In [274]:
arr=np.concatenate((ans_vec, ques_vec), axis=0)

In [277]:
#apply word embedding
# data["summary"] = data["summary"].map(
#     lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pprint import pprint

vocab_size = 80000
max_len = 30


# build vocabulary from training set
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(arr)

def _preprocess(list_of_text):
    return pad_sequences(
        tokenizer.texts_to_sequences(list_of_text),
        maxlen=max_len,
        padding='post',
      
    )
    

# padding is done inside: 
corpus_ans = _preprocess(ans_vec)
corpus_ques = _preprocess(ques_vec)

print(corpus_ans)

[[ 190    0    0 ...    0    0    0]
 [3889 6853 2719 ...  262  116  509]
 [1911 1534  320 ...    0    0    0]
 ...
 [  34   29   33 ...   34   29  574]
 [  24 1267  135 ... 2158  111 1241]
 [ 594 3069  611 ...    0    0    0]]


In [198]:
from gensim.corpora import Dictionary
dictionary = Dictionary(arr)
dictionary.filter_extremes(no_below=20, no_above=0.5)
corpus_ans = [dictionary.doc2bow(doc) for doc in ans_vec]
corpus_ques = [dictionary.doc2bow(doc) for doc in ques_vec]

In [ ]:
corpus_ques, corpus_ans

In [200]:
# vect=np.concatenate((corpus_ques, corpus_ans), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [251]:
train_ans=corpus_ans[:2208]
test_ans=corpus_ans[2208:]
train_ques=corpus_ques[:2208]
test_ques=corpus_ques[2208:]
test_ans#.shape,train_ans.shape,test_ques.shape,train_ques.shape

[[(12, 1),
  (271, 1),
  (283, 1),
  (285, 1),
  (378, 1),
  (414, 1),
  (489, 1),
  (518, 1),
  (971, 1),
  (1041, 1),
  (1060, 1),
  (1131, 1),
  (1440, 1)],
 [(17, 1),
  (41, 1),
  (70, 1),
  (113, 1),
  (249, 1),
  (265, 1),
  (273, 1),
  (329, 1),
  (511, 1),
  (670, 1),
  (764, 1),
  (970, 1),
  (1060, 1),
  (1651, 1)],
 [(11, 1),
  (24, 1),
  (38, 1),
  (46, 1),
  (48, 1),
  (82, 1),
  (93, 3),
  (107, 1),
  (113, 1),
  (118, 3),
  (132, 1),
  (174, 1),
  (209, 2),
  (232, 1),
  (236, 2),
  (249, 1),
  (265, 1),
  (271, 3),
  (283, 1),
  (300, 1),
  (301, 1),
  (302, 1),
  (303, 1),
  (309, 1),
  (319, 2),
  (325, 1),
  (340, 2),
  (345, 1),
  (414, 1),
  (416, 1),
  (437, 2),
  (487, 1),
  (494, 1),
  (510, 1),
  (623, 1),
  (811, 1),
  (843, 1),
  (909, 1),
  (929, 1),
  (950, 1),
  (962, 2),
  (1024, 1),
  (1140, 1),
  (1192, 1),
  (1436, 1),
  (1528, 1),
  (1755, 1)],
 [(22, 2),
  (26, 2),
  (36, 4),
  (46, 1),
  (53, 1),
  (67, 1),
  (76, 1),
  (82, 1),
  (85, 1),
  (90, 1)

In [253]:
train_ques = [i.split() for i in train_ques]
train_ans = [i.split() for i in train_ans]

AttributeError: ignored

In [254]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

df_all = pd.DataFrame({'question':train_ques, 
                       'answer':train_ans})

transformer = FeatureUnion([
                ('question', 
                  Pipeline([('extract_field',
                              FunctionTransformer(lambda x: x['question'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('answer', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['answer'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())]))]) 

transformer.fit(df_all)

search_vocab = transformer.transformer_list[0][1].steps[1][1].get_feature_names() 
product_vocab = transformer.transformer_list[1][1].steps[1][1].get_feature_names()
vocab = search_vocab + product_vocab

print(vocab)
print(transformer.transform(df_all).toarray())

AttributeError: ignored

In [237]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None)
tfidf_matrix = tfidf.fit_transform(train_ques + train_ans) # This line is the issue

# tfidf.fit(vect)
# dataset_ans = tfidf.transform(corpus_ans)
# dataset_ques = tfidf.transform(corpus_ques)

In [238]:
# dataset_ans,dataset_ques
tfidf_matrix

<4416x6173 sparse matrix of type '<class 'numpy.float64'>'
	with 137261 stored elements in Compressed Sparse Row format>

In [246]:
from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()
le2.fit(pd.Series(data_set['Judge_score']).values)
data_set['Judge_score'] = le2.transform(data_set['Judge_score'].values)

In [245]:
data_set.shape

(3281, 3)

In [247]:
y=dataset['Judge_score'][:2208]

In [248]:
# train_data=pd.DataFrame()
# test_data=pd.DataFrame()
# train_data=train_ques,train_ans
# test_data=test_ques,test_ans

In [249]:
# train_data

In [250]:
from sklearn import svm
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, y, train_size = 0.8, stratify = y, random_state = 2022)
model = svm.LinearSVC(verbose=True)   #Checks also SVC
model.fit(X_train, y_train)

ValueError: ignored

In [ ]:
preds = model.predict(X_val)

In [ ]:
accuracy_score(y_val, preds)